In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/email-spam-classification-dataset-csv/emails.csv


In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix 

# Create a DataFrame 

In [3]:
df = pd.read_csv('/kaggle/input/email-spam-classification-dataset-csv/emails.csv')

1 = spam, 0 = not spam

In [4]:
df

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,Email 5168,2,2,2,3,0,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,Email 5169,35,27,11,2,6,5,151,4,3,...,0,0,0,0,0,0,0,1,0,0
5169,Email 5170,0,0,1,1,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,1
5170,Email 5171,2,7,1,0,2,1,28,2,0,...,0,0,0,0,0,0,0,1,0,1


# DataFrame 
* a dataframe is 2D data structure with columns 
* describe() is a method that returns the stats of a dataframe including its count, mean, standard deviation etc 

In [13]:
df.describe(include='object')t

,Email No.
count,5172
unique,5172
top,Email 1
freq,1


In [14]:
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + DF.TARGET + '; ANC1: ' + df.anchor 

AttributeError: 'DataFrame' object has no attribute 'context'

# Tokenization 

Datasets is a library by HuggingFace that allows easily accessing and sharing datasets for Audio, Computer Vision, and NLP tasks. 

# Pandas Data Frame vs HF Dataset 
* Pandas DataFrame - 2D labeled data structure w columns of potentially different types 
* HF Dataset - library and data structure to store and access datasets 

In [5]:
from datasets import Dataset, DatasetDict

#converts a pandas DataFrame into a Hugging Face Dataset object 
ds = Dataset.from_pandas(df)

In [6]:
ds

Dataset({
    features: ['Email No.', 'the', 'to', 'ect', 'and', 'for', 'of', 'a', 'you', 'hou', 'in', 'on', 'is', 'this', 'enron', 'i', 'be', 'that', 'will', 'have', 'with', 'your', 'at', 'we', 's', 'are', 'it', 'by', 'com', 'as', 'from', 'gas', 'or', 'not', 'me', 'deal', 'if', 'meter', 'hpl', 'please', 're', 'e', 'any', 'our', 'corp', 'can', 'd', 'all', 'has', 'was', 'know', 'need', 'an', 'forwarded', 'new', 't', 'may', 'up', 'j', 'mmbtu', 'should', 'do', 'am', 'get', 'out', 'see', 'no', 'there', 'price', 'daren', 'but', 'been', 'company', 'l', 'these', 'let', 'so', 'would', 'm', 'into', 'xls', 'farmer', 'attached', 'us', 'information', 'they', 'message', 'day', 'time', 'my', 'one', 'what', 'only', 'http', 'th', 'volume', 'mail', 'contract', 'which', 'month', 'more', 'robert', 'sitara', 'about', 'texas', 'nom', 'energy', 'pec', 'questions', 'www', 'deals', 'volumes', 'pm', 'ena', 'now', 'their', 'file', 'some', 'email', 'just', 'also', 'call', 'change', 'other', 'here', 'like', 'b', 

This is a model developed by Microsoft called DeBERTa. It's the third version and has fewer parameters than the 'base' or 'large versions. 

In [7]:
model_nm = 'microsoft/deberta-v3-small'

# Transformers and AutoTokenizer 

* transformers - HF library with wide range of pre-trained models for NLP 
* AutoModelForSequenceClassification - loads appropriate model architecture 
* AutoTokenizer - loads appropriate tokenizer (processes text so that model can understand) 
* tokenizer splits the input text into tokens --> numbers --> adds any special tokens 

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
tokz = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
tokz.tokenize("Hi my name is Natalie.")

['▁Hi', '▁my', '▁name', '▁is', '▁Natalie', '.']

In [10]:
tokz('Hello my name is Nat')

{'input_ids': [1, 5365, 312, 601, 269, 12773, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [11]:
def tok_func(x): return tokz(x['input'])

Batch mapping allows you to speed up processing and freely control the size of the generated dataset. HF's Tokenizers library works faster with batches because it parallelizes the tokenization of all the examples in a batch. 

In [12]:
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/5172 [00:00<?, ? examples/s]

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:
row = 